In [125]:
import numpy as np
import tqdm.notebook as tqdm
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
import tqdm.notebook as tqdm

import warnings
warnings.filterwarnings('ignore') 

In [126]:
from sklearn.datasets import fetch_covtype
x, y = fetch_covtype(return_X_y=True)
x = KBinsDiscretizer(encode='onehot-dense').fit_transform(x)

x, x_test, y, y_test = train_test_split(x, y, random_state=42, stratify=y)

In [102]:
class NBC:
    """Naive Bayes Classifier."""

    def __init__(self):
        """Get a new classifier."""
        self._is_fit = False
        self.priors = None
        self.likelihoods = None
        self.classes = None


    def fit(self, x, y, reg=1.0):
        """Fit the model."""
        self.classes = classes = np.unique(y)
        self.priors = [(y == c).sum() / len(y) for c in classes]
        
        x_by_class = [x[y == c] for c in classes]
        counts = np.array([q.sum(axis=0) for q in x_by_class]) + reg
        self.likelihoods = counts / counts.sum(axis=1).reshape(-1, 1)  # P(x_i | y)
        self._is_fit = True
        return self
        
    def _predict_item(self, x):
        if not self._is_fit:
            raise RuntimeError("Model not fit.")

        probs = (likelihoods ** x).prod(axis=1) * priors
        probs = probs / probs.sum().reshape(-1, 1)
        return probs
        
    def predict(self, x):
        return np.vstack([self._predict_item(q) for q in tqdm.tqdm(x)])

In [103]:
clf = NBC().fit(x, y)

In [123]:
pred = clf.predict(x)
pred = pred.argmax(axis=1)
print(classification_report(y_true=y, y_pred=pred))

  0%|          | 0/435759 [00:00<?, ?it/s]